# Random forest classification

## Single-node scikit-learn

<table>
    <tr>
        <td>
            <img src="../_img/scikit-learn.png" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="450">
        </td>
    </tr>
</table>

In [ ]:
numeric_feat = [
    "pickup_weekday",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "high_tip"

# Load data and feature engineering

Load a full month for this exercise

In [ ]:
import os
import pandas as pd

import snowflake.connector

SNOWFLAKE_ACCOUNT = os.environ["SNOWFLAKE_ACCOUNT"]
SNOWFLAKE_USER = os.environ["SNOWFLAKE_USER"]
SNOWFLAKE_PASSWORD = os.environ["SNOWFLAKE_PASSWORD"]

SNOWFLAKE_WAREHOUSE = os.environ["SNOWFLAKE_WAREHOUSE"]
TAXI_DATABASE = os.environ["TAXI_DATABASE"]
TAXI_SCHEMA = os.environ["TAXI_SCHEMA"]

conn_info = {
    "account": SNOWFLAKE_ACCOUNT,
    "user": SNOWFLAKE_USER,
    "password": SNOWFLAKE_PASSWORD,
    "warehouse": SNOWFLAKE_WAREHOUSE,
    "database": TAXI_DATABASE,
    "schema": TAXI_SCHEMA,
}
conn = snowflake.connector.connect(**conn_info)

In [ ]:
query = """
SELECT
    pickup_taxizone_id,
    dropoff_taxizone_id,
    passenger_count,
    DIV0(tip_amount, fare_amount) > 0.2 AS high_tip,
    DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
    WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
    HOUR(pickup_datetime) AS pickup_hour,
    (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
    MINUTE(pickup_datetime) AS pickup_minute
FROM taxi_yellow
WHERE
    DATE_TRUNC('MONTH', pickup_datetime) = %s
"""
taxi = conn.cursor().execute(query, "2019-01-01")
columns = [x[0] for x in taxi.description]
taxi = pd.DataFrame(taxi.fetchall(), columns=columns)
taxi.columns = taxi.columns.str.lower()

In [ ]:
print(f"Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB")

In [ ]:
taxi_train = taxi[features + [y_col]]

In [ ]:
taxi_train.high_tip.value_counts()

In [ ]:
taxi_train.head()

# Train model

Setting `n_jobs=-1` tells scikit-learn to use all available cores on this machine to train models. Note that scikit-learn does NOT use the GPU, its using CPU cores here.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

In [ ]:
%%time
_ = rfc.fit(taxi_train[features], taxi_train[y_col])

## Save model

In [ ]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/random_forest_scikit.pkl", "wb") as f:
    cloudpickle.dump(rfc, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi = conn.cursor().execute(query, "2019-02-01")
columns = [x[0] for x in taxi.description]
taxi = pd.DataFrame(taxi.fetchall(), columns=columns)
taxi.columns = taxi.columns.str.lower()
taxi_test = taxi

In [ ]:
from sklearn.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[:, 1]
roc_auc_score(taxi_test[y_col], preds)